In [ ]:
# @title <p>Essential Import
import os, shutil, json
from PIL import Image
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, random as rd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title <p>Torch Essential Import
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ResnetBlock(nn.Module):
  def __init__(self, in_ch, inter_ch, identity_downsample=None, stride=1):
    super().__init__()
    self.conv1 = nn.Conv2d(in_ch, inter_ch, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn1 = nn.BatchNorm2d(inter_ch)
    self.conv2 = nn.Conv2d(inter_ch, inter_ch, kernel_size=3, stride=stride, padding=1, bias=False)
    self.conv3 = nn.Conv2d(inter_ch, inter_ch*4, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn2 = nn.BatchNorm2d(inter_ch*4)
    self.relu = nn.ReLU()
    self.identity_downsample = identity_downsample

  def forward(self, x):
    identity = x.clone()

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bn2(x)

    if self.identity_downsample is not None:
      identity = self.identity_downsample(identity)

    x += identity
    x = self.relu(x)

    return x

class ResnetModel(nn.Module):
  def __init__(self, model_type="ResNet50", image_channel=3, num_classes=1000):
    super().__init__()
    self.in_ch = 64
    self.conv1 = nn.Conv2d(image_channel, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    if model_type == "ResNet50":
      self.config = [3, 4, 6, 3]

    elif model_type == "ResNet101":
      self.config = [3, 4, 23, 3]

    elif model_type == "ResNet152":
      self.config = [3, 8, 36, 3]

    self.layer1 = self.make_resnet_block(self.config[0], inter_ch=64, stride=1)
    self.layer2 = self.make_resnet_block(self.config[1], inter_ch=128, stride=2)
    self.layer3 = self.make_resnet_block(self.config[2], inter_ch=256, stride=2)
    self.layer4 = self.make_resnet_block(self.config[3], inter_ch=512, stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512 * 4, num_classes)

  def make_resnet_block(self, num_block, inter_ch, stride):
    identity_downsample = None
    layers = []

    if stride != 1 or self.in_ch != inter_ch * 4:
      identity_downsample = nn.Sequential(
          nn.Conv2d(self.in_ch, inter_ch*4, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(inter_ch*4)
      )

    layers.append(ResnetBlock(self.in_ch, inter_ch, identity_downsample, stride))

    self.in_ch = inter_ch*4

    for i in range(num_block-1):
      layers.append(ResnetBlock(self.in_ch, inter_ch))

    return nn.Sequential(*layers)

  def forward(self, x):

    x = self.conv1(x)
    x = self.bn(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)

    return x

torch.Size([1, 1000])